In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 32
tree_depth = 12
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.161576271057129 | KNN Loss: 6.226352691650391 | BCE Loss: 1.9352238178253174
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.159000396728516 | KNN Loss: 6.226648807525635 | BCE Loss: 1.932352066040039
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.188031196594238 | KNN Loss: 6.226075172424316 | BCE Loss: 1.9619560241699219
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.213977813720703 | KNN Loss: 6.225383758544922 | BCE Loss: 1.9885936975479126
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.180749893188477 | KNN Loss: 6.225130081176758 | BCE Loss: 1.9556200504302979
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.228500366210938 | KNN Loss: 6.224370956420898 | BCE Loss: 2.004129409790039
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.17105484008789 | KNN Loss: 6.22470760345459 | BCE Loss: 1.9463468790054321
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.153498649597168 | KNN Loss: 6.224190711975098 | BCE Loss: 1.929308056

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 6.008642196655273 | KNN Loss: 4.881716251373291 | BCE Loss: 1.1269259452819824
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 5.880500793457031 | KNN Loss: 4.756161212921143 | BCE Loss: 1.1243395805358887
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 5.784660816192627 | KNN Loss: 4.677248954772949 | BCE Loss: 1.1074117422103882
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 5.713019847869873 | KNN Loss: 4.59433126449585 | BCE Loss: 1.118688702583313
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 5.579710006713867 | KNN Loss: 4.452706813812256 | BCE Loss: 1.1270031929016113
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 5.4775896072387695 | KNN Loss: 4.382556438446045 | BCE Loss: 1.0950329303741455
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 5.459367752075195 | KNN Loss: 4.336392402648926 | BCE Loss: 1.1229753494262695
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 5.407453536987305 | KNN Loss: 4.294209957122803 | BCE Loss:

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 4.889537811279297 | KNN Loss: 3.8321566581726074 | BCE Loss: 1.0573813915252686
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 4.884041786193848 | KNN Loss: 3.817591905593872 | BCE Loss: 1.0664498805999756
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 4.904548645019531 | KNN Loss: 3.816315174102783 | BCE Loss: 1.0882335901260376
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 4.870769023895264 | KNN Loss: 3.8403100967407227 | BCE Loss: 1.030458927154541
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 4.880009651184082 | KNN Loss: 3.8069984912872314 | BCE Loss: 1.0730112791061401
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 4.868500709533691 | KNN Loss: 3.8365886211395264 | BCE Loss: 1.0319123268127441
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 4.897292613983154 | KNN Loss: 3.8595547676086426 | BCE Loss: 1.0377379655838013
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 4.877297401428223 | KNN Loss: 3.8279037475585938 | BC

Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 4.769382953643799 | KNN Loss: 3.7356760501861572 | BCE Loss: 1.0337069034576416
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 4.821622848510742 | KNN Loss: 3.7641007900238037 | BCE Loss: 1.0575222969055176
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 4.8612380027771 | KNN Loss: 3.805204153060913 | BCE Loss: 1.0560338497161865
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 4.810745716094971 | KNN Loss: 3.7756741046905518 | BCE Loss: 1.0350717306137085
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 4.864107608795166 | KNN Loss: 3.8003976345062256 | BCE Loss: 1.0637099742889404
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 4.850316524505615 | KNN Loss: 3.797315835952759 | BCE Loss: 1.0530006885528564
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 4.829188346862793 | KNN Loss: 3.7873759269714355 | BCE Loss: 1.0418126583099365
Epoch 33 / 500 | iteration 20 / 30 | Total Loss: 4.849262237548828 | KNN Loss: 3.7826976776123047 | BCE

Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 4.815564155578613 | KNN Loss: 3.7677969932556152 | BCE Loss: 1.0477674007415771
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 4.830121994018555 | KNN Loss: 3.8136463165283203 | BCE Loss: 1.0164759159088135
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 4.812733173370361 | KNN Loss: 3.7600300312042236 | BCE Loss: 1.0527031421661377
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 4.761682510375977 | KNN Loss: 3.7450225353240967 | BCE Loss: 1.0166598558425903
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 4.831888198852539 | KNN Loss: 3.7563159465789795 | BCE Loss: 1.0755720138549805
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 4.798451900482178 | KNN Loss: 3.7779626846313477 | BCE Loss: 1.02048921585083
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 4.813273906707764 | KNN Loss: 3.7830395698547363 | BCE Loss: 1.030234456062317
Epoch 44 / 500 | iteration 15 / 30 | Total Loss: 4.818156719207764 | KNN Loss: 3.7547125816345215 | BC

...

Epoch 448 / 500 | iteration 25 / 30 | Total Loss: 4.692770481109619 | KNN Loss: 3.685917615890503 | BCE Loss: 1.0068527460098267
Epoch   449: reducing learning rate of group 0 to 1.1270e-07.
Epoch 449 / 500 | iteration 0 / 30 | Total Loss: 4.7409772872924805 | KNN Loss: 3.6936752796173096 | BCE Loss: 1.04730224609375
Epoch 449 / 500 | iteration 5 / 30 | Total Loss: 4.7634100914001465 | KNN Loss: 3.726328134536743 | BCE Loss: 1.0370819568634033
Epoch 449 / 500 | iteration 10 / 30 | Total Loss: 4.75009822845459 | KNN Loss: 3.7112393379211426 | BCE Loss: 1.0388586521148682
Epoch 449 / 500 | iteration 15 / 30 | Total Loss: 4.666780471801758 | KNN Loss: 3.674314498901367 | BCE Loss: 0.9924658536911011
Epoch 449 / 500 | iteration 20 / 30 | Total Loss: 4.720156192779541 | KNN Loss: 3.6891753673553467 | BCE Loss: 1.0309808254241943
Epoch 449 / 500 | iteration 25 / 30 | Total Loss: 4.714337348937988 | KNN Loss: 3.6602962017059326 | BCE Loss: 1.0540409088134766
Epoch 450 / 500 | iteration 0 / 30

Epoch 459 / 500 | iteration 15 / 30 | Total Loss: 4.792212963104248 | KNN Loss: 3.7266476154327393 | BCE Loss: 1.0655654668807983
Epoch 459 / 500 | iteration 20 / 30 | Total Loss: 4.684537410736084 | KNN Loss: 3.6626217365264893 | BCE Loss: 1.0219155550003052
Epoch 459 / 500 | iteration 25 / 30 | Total Loss: 4.727397441864014 | KNN Loss: 3.6909310817718506 | BCE Loss: 1.036466360092163
Epoch   460: reducing learning rate of group 0 to 7.8888e-08.
Epoch 460 / 500 | iteration 0 / 30 | Total Loss: 4.748035430908203 | KNN Loss: 3.686722993850708 | BCE Loss: 1.0613125562667847
Epoch 460 / 500 | iteration 5 / 30 | Total Loss: 4.705294609069824 | KNN Loss: 3.6800954341888428 | BCE Loss: 1.0251991748809814
Epoch 460 / 500 | iteration 10 / 30 | Total Loss: 4.7114105224609375 | KNN Loss: 3.686742067337036 | BCE Loss: 1.0246682167053223
Epoch 460 / 500 | iteration 15 / 30 | Total Loss: 4.734474182128906 | KNN Loss: 3.681114912033081 | BCE Loss: 1.0533595085144043
Epoch 460 / 500 | iteration 20 / 

Epoch 470 / 500 | iteration 5 / 30 | Total Loss: 4.760499000549316 | KNN Loss: 3.741896629333496 | BCE Loss: 1.0186023712158203
Epoch 470 / 500 | iteration 10 / 30 | Total Loss: 4.7106242179870605 | KNN Loss: 3.6862051486968994 | BCE Loss: 1.0244190692901611
Epoch 470 / 500 | iteration 15 / 30 | Total Loss: 4.741048812866211 | KNN Loss: 3.709479808807373 | BCE Loss: 1.031569004058838
Epoch 470 / 500 | iteration 20 / 30 | Total Loss: 4.735234260559082 | KNN Loss: 3.6780471801757812 | BCE Loss: 1.0571868419647217
Epoch 470 / 500 | iteration 25 / 30 | Total Loss: 4.744417190551758 | KNN Loss: 3.7214269638061523 | BCE Loss: 1.0229899883270264
Epoch   471: reducing learning rate of group 0 to 5.5221e-08.
Epoch 471 / 500 | iteration 0 / 30 | Total Loss: 4.764540672302246 | KNN Loss: 3.7196900844573975 | BCE Loss: 1.0448505878448486
Epoch 471 / 500 | iteration 5 / 30 | Total Loss: 4.7561492919921875 | KNN Loss: 3.7200989723205566 | BCE Loss: 1.0360503196716309
Epoch 471 / 500 | iteration 10 /

Epoch 480 / 500 | iteration 25 / 30 | Total Loss: 4.724907875061035 | KNN Loss: 3.7121329307556152 | BCE Loss: 1.0127747058868408
Epoch 481 / 500 | iteration 0 / 30 | Total Loss: 4.679256439208984 | KNN Loss: 3.6544671058654785 | BCE Loss: 1.0247893333435059
Epoch 481 / 500 | iteration 5 / 30 | Total Loss: 4.726641654968262 | KNN Loss: 3.69728946685791 | BCE Loss: 1.0293521881103516
Epoch 481 / 500 | iteration 10 / 30 | Total Loss: 4.739554405212402 | KNN Loss: 3.729313373565674 | BCE Loss: 1.0102410316467285
Epoch 481 / 500 | iteration 15 / 30 | Total Loss: 4.686199188232422 | KNN Loss: 3.6696643829345703 | BCE Loss: 1.0165349245071411
Epoch 481 / 500 | iteration 20 / 30 | Total Loss: 4.730989933013916 | KNN Loss: 3.6781208515167236 | BCE Loss: 1.0528690814971924
Epoch 481 / 500 | iteration 25 / 30 | Total Loss: 4.7508225440979 | KNN Loss: 3.6973724365234375 | BCE Loss: 1.053450107574463
Epoch   482: reducing learning rate of group 0 to 3.8655e-08.
Epoch 482 / 500 | iteration 0 / 30 |

Epoch 491 / 500 | iteration 15 / 30 | Total Loss: 4.682605266571045 | KNN Loss: 3.6712288856506348 | BCE Loss: 1.0113763809204102
Epoch 491 / 500 | iteration 20 / 30 | Total Loss: 4.703763008117676 | KNN Loss: 3.685659408569336 | BCE Loss: 1.0181033611297607
Epoch 491 / 500 | iteration 25 / 30 | Total Loss: 4.735307693481445 | KNN Loss: 3.7150192260742188 | BCE Loss: 1.0202887058258057
Epoch 492 / 500 | iteration 0 / 30 | Total Loss: 4.648627758026123 | KNN Loss: 3.6306040287017822 | BCE Loss: 1.0180238485336304
Epoch 492 / 500 | iteration 5 / 30 | Total Loss: 4.706360816955566 | KNN Loss: 3.6899285316467285 | BCE Loss: 1.016432523727417
Epoch 492 / 500 | iteration 10 / 30 | Total Loss: 4.772653102874756 | KNN Loss: 3.7352843284606934 | BCE Loss: 1.0373687744140625
Epoch 492 / 500 | iteration 15 / 30 | Total Loss: 4.7292914390563965 | KNN Loss: 3.6734671592712402 | BCE Loss: 1.0558241605758667
Epoch 492 / 500 | iteration 20 / 30 | Total Loss: 4.723285675048828 | KNN Loss: 3.70104026794

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 2.8700,  3.2148,  2.5888,  3.2199,  2.9580,  0.7435,  2.5452,  1.7208,
          2.3905,  2.0455,  2.3058,  1.8545,  0.7572,  1.9303,  1.3485,  1.4854,
          2.4915,  2.9553,  2.8168,  1.8230,  1.8092,  2.7377,  2.3335,  2.2920,
          2.6009,  1.8221,  2.0219,  1.3945,  1.5464,  0.3820, -0.2454,  0.9553,
          0.2409,  1.0272,  1.3222,  1.4977,  1.0836,  2.8201,  0.8823,  1.3845,
          0.9177, -0.6213, -0.2419,  2.4166,  2.2063,  0.6436, -0.1784, -0.0311,
          1.5638,  2.1203,  1.9193,  0.2048,  1.4929,  0.5505, -0.6681,  1.1283,
          1.5035,  0.8375,  1.4346,  1.8754,  0.6843,  0.9019,  0.2014,  1.7707,
          1.2772,  1.5920, -2.0957,  0.3177,  2.0157,  1.9995,  2.4371,  0.5219,
          1.3739,  2.4774,  2.0058,  1.2663,  0.2416,  0.7286,  0.2631,  1.6594,
          0.0389,  0.4202,  1.9244, -0.2867,  0.2105, -1.0360, -2.3141, -0.4303,
          0.5728, -1.8165,  0.4920, -0.1380, -0.5948, -0.8402,  0.5739,  1.3905,
         -0.8045, -0.6958,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [10]:
dataset_ = [d[0].cpu() for d in dataset]

In [11]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 78.14it/s]


In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [13]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [14]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [16]:
tensor_dataset = torch.stack(dataset_)

In [17]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [18]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [19]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [20]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [21]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [22]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [23]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [24]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [25]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [26]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [27]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [28]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [29]:
losses = []
accs = []
sparsity = []

In [30]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
layer 7: 0.0
layer 8: 0.0
layer 9: 0.0
layer 10: 0.0
Epoch: 00 | Batch: 000 / 026 | Total loss: 9.626 | Reg loss: 0.014 | Tree loss: 9.626 | Accuracy: 0.000000 | 6.821 sec/iter
Epoch: 00 | Batch: 001 / 026 | Total loss: 9.624 | Reg loss: 0.013 | Tree loss: 9.624 | Accuracy: 0.000000 | 6.117 sec/iter
Epoch: 00 | Batch: 002 / 026 | Total loss: 9.623 | Reg loss: 0.012 | Tree loss: 9.623 | Accuracy: 0.000000 | 5.831 sec/iter
Epoch: 00 | Batch: 003 / 026 | Total loss: 9.621 | Reg loss: 0.011 | Tree loss: 9.621 | Accuracy: 0.000000 | 5.674 sec/iter
Epoch: 00 | Batch: 004 / 026 | Total loss: 9.619 | Reg loss: 0.010 | Tree loss: 9.619 | Accuracy: 0.000000 | 5.561 sec/iter
Epoch: 00 | Batch: 005 / 026 | Total loss: 9.617 | Reg loss: 0.009 | Tree loss: 9.617 | Accuracy: 0.000000 | 5.479 sec/iter
Epoch: 00 | Batch: 006 / 026 | Total loss: 9.616 | Reg loss: 0.008 | Tree loss: 9.616 | 

Epoch: 02 | Batch: 008 / 026 | Total loss: 9.578 | Reg loss: 0.006 | Tree loss: 9.578 | Accuracy: 0.111328 | 5.041 sec/iter
Epoch: 02 | Batch: 009 / 026 | Total loss: 9.579 | Reg loss: 0.006 | Tree loss: 9.579 | Accuracy: 0.115234 | 5.037 sec/iter
Epoch: 02 | Batch: 010 / 026 | Total loss: 9.576 | Reg loss: 0.007 | Tree loss: 9.576 | Accuracy: 0.138672 | 5.03 sec/iter
Epoch: 02 | Batch: 011 / 026 | Total loss: 9.575 | Reg loss: 0.007 | Tree loss: 9.575 | Accuracy: 0.132812 | 5.015 sec/iter
Epoch: 02 | Batch: 012 / 026 | Total loss: 9.573 | Reg loss: 0.007 | Tree loss: 9.573 | Accuracy: 0.132812 | 5.026 sec/iter
Epoch: 02 | Batch: 013 / 026 | Total loss: 9.573 | Reg loss: 0.007 | Tree loss: 9.573 | Accuracy: 0.134766 | 5.038 sec/iter
Epoch: 02 | Batch: 014 / 026 | Total loss: 9.572 | Reg loss: 0.007 | Tree loss: 9.572 | Accuracy: 0.130859 | 5.051 sec/iter
Epoch: 02 | Batch: 015 / 026 | Total loss: 9.570 | Reg loss: 0.008 | Tree loss: 9.570 | Accuracy: 0.138672 | 5.064 sec/iter
Epoch: 02

Epoch: 04 | Batch: 017 / 026 | Total loss: 9.478 | Reg loss: 0.014 | Tree loss: 9.478 | Accuracy: 0.130859 | 5.113 sec/iter
Epoch: 04 | Batch: 018 / 026 | Total loss: 9.483 | Reg loss: 0.015 | Tree loss: 9.483 | Accuracy: 0.111328 | 5.115 sec/iter
Epoch: 04 | Batch: 019 / 026 | Total loss: 9.474 | Reg loss: 0.015 | Tree loss: 9.474 | Accuracy: 0.099609 | 5.117 sec/iter
Epoch: 04 | Batch: 020 / 026 | Total loss: 9.463 | Reg loss: 0.016 | Tree loss: 9.463 | Accuracy: 0.105469 | 5.117 sec/iter
Epoch: 04 | Batch: 021 / 026 | Total loss: 9.454 | Reg loss: 0.016 | Tree loss: 9.454 | Accuracy: 0.105469 | 5.116 sec/iter
Epoch: 04 | Batch: 022 / 026 | Total loss: 9.450 | Reg loss: 0.016 | Tree loss: 9.450 | Accuracy: 0.111328 | 5.114 sec/iter
Epoch: 04 | Batch: 023 / 026 | Total loss: 9.445 | Reg loss: 0.017 | Tree loss: 9.445 | Accuracy: 0.095703 | 5.111 sec/iter
Epoch: 04 | Batch: 024 / 026 | Total loss: 9.430 | Reg loss: 0.017 | Tree loss: 9.430 | Accuracy: 0.058594 | 5.109 sec/iter
Epoch: 0

Average sparseness: 0.9821428571428572
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
layer 5: 0.9821428571428571
layer 6: 0.982142857142857
layer 7: 0.9821428571428573
layer 8: 0.9821428571428573
layer 9: 0.9821428571428573
layer 10: 0.9821428571428573
Epoch: 07 | Batch: 000 / 026 | Total loss: 9.285 | Reg loss: 0.017 | Tree loss: 9.285 | Accuracy: 0.128906 | 5.137 sec/iter
Epoch: 07 | Batch: 001 / 026 | Total loss: 9.248 | Reg loss: 0.017 | Tree loss: 9.248 | Accuracy: 0.109375 | 5.139 sec/iter
Epoch: 07 | Batch: 002 / 026 | Total loss: 9.236 | Reg loss: 0.017 | Tree loss: 9.236 | Accuracy: 0.107422 | 5.14 sec/iter
Epoch: 07 | Batch: 003 / 026 | Total loss: 9.213 | Reg loss: 0.017 | Tree loss: 9.213 | Accuracy: 0.097656 | 5.14 sec/iter
Epoch: 07 | Batch: 004 / 026 | Total loss: 9.208 | Reg loss: 0.017 | Tree loss: 9.208 | Accuracy: 0.103516 | 5.141 sec/iter
Epoch: 07 | Batch: 005 / 026 | Tota

Epoch: 09 | Batch: 006 / 026 | Total loss: 8.710 | Reg loss: 0.021 | Tree loss: 8.710 | Accuracy: 0.056641 | 5.129 sec/iter
Epoch: 09 | Batch: 007 / 026 | Total loss: 8.681 | Reg loss: 0.021 | Tree loss: 8.681 | Accuracy: 0.058594 | 5.128 sec/iter
Epoch: 09 | Batch: 008 / 026 | Total loss: 8.633 | Reg loss: 0.022 | Tree loss: 8.633 | Accuracy: 0.066406 | 5.124 sec/iter
Epoch: 09 | Batch: 009 / 026 | Total loss: 8.592 | Reg loss: 0.022 | Tree loss: 8.592 | Accuracy: 0.064453 | 5.126 sec/iter
Epoch: 09 | Batch: 010 / 026 | Total loss: 8.551 | Reg loss: 0.022 | Tree loss: 8.551 | Accuracy: 0.062500 | 5.128 sec/iter
Epoch: 09 | Batch: 011 / 026 | Total loss: 8.539 | Reg loss: 0.022 | Tree loss: 8.539 | Accuracy: 0.062500 | 5.13 sec/iter
Epoch: 09 | Batch: 012 / 026 | Total loss: 8.552 | Reg loss: 0.023 | Tree loss: 8.552 | Accuracy: 0.066406 | 5.132 sec/iter
Epoch: 09 | Batch: 013 / 026 | Total loss: 8.505 | Reg loss: 0.023 | Tree loss: 8.505 | Accuracy: 0.070312 | 5.135 sec/iter
Epoch: 09

...

Epoch: 88 | Batch: 009 / 026 | Total loss: 2.935 | Reg loss: 0.032 | Tree loss: 2.935 | Accuracy: 0.113281 | 4.865 sec/iter
Epoch: 88 | Batch: 010 / 026 | Total loss: 2.959 | Reg loss: 0.032 | Tree loss: 2.959 | Accuracy: 0.105469 | 4.865 sec/iter
Epoch: 88 | Batch: 011 / 026 | Total loss: 2.993 | Reg loss: 0.032 | Tree loss: 2.993 | Accuracy: 0.105469 | 4.864 sec/iter
Epoch: 88 | Batch: 012 / 026 | Total loss: 2.934 | Reg loss: 0.032 | Tree loss: 2.934 | Accuracy: 0.119141 | 4.864 sec/iter
Epoch: 88 | Batch: 013 / 026 | Total loss: 2.936 | Reg loss: 0.032 | Tree loss: 2.936 | Accuracy: 0.123047 | 4.863 sec/iter
Epoch: 88 | Batch: 014 / 026 | Total loss: 2.921 | Reg loss: 0.032 | Tree loss: 2.921 | Accuracy: 0.103516 | 4.863 sec/iter
Epoch: 88 | Batch: 015 / 026 | Total loss: 2.917 | Reg loss: 0.032 | Tree loss: 2.917 | Accuracy: 0.134766 | 4.862 sec/iter
Epoch: 88 | Batch: 016 / 026 | Total loss: 2.951 | Reg loss: 0.032 | Tree loss: 2.951 | Accuracy: 0.101562 | 4.862 sec/iter
Epoch: 8

Epoch: 90 | Batch: 018 / 026 | Total loss: 2.915 | Reg loss: 0.032 | Tree loss: 2.915 | Accuracy: 0.148438 | 4.84 sec/iter
Epoch: 90 | Batch: 019 / 026 | Total loss: 2.913 | Reg loss: 0.032 | Tree loss: 2.913 | Accuracy: 0.111328 | 4.84 sec/iter
Epoch: 90 | Batch: 020 / 026 | Total loss: 2.905 | Reg loss: 0.032 | Tree loss: 2.905 | Accuracy: 0.121094 | 4.839 sec/iter
Epoch: 90 | Batch: 021 / 026 | Total loss: 2.882 | Reg loss: 0.032 | Tree loss: 2.882 | Accuracy: 0.144531 | 4.839 sec/iter
Epoch: 90 | Batch: 022 / 026 | Total loss: 2.862 | Reg loss: 0.032 | Tree loss: 2.862 | Accuracy: 0.152344 | 4.838 sec/iter
Epoch: 90 | Batch: 023 / 026 | Total loss: 2.926 | Reg loss: 0.032 | Tree loss: 2.926 | Accuracy: 0.111328 | 4.838 sec/iter
Epoch: 90 | Batch: 024 / 026 | Total loss: 2.883 | Reg loss: 0.032 | Tree loss: 2.883 | Accuracy: 0.123047 | 4.838 sec/iter
Epoch: 90 | Batch: 025 / 026 | Total loss: 2.868 | Reg loss: 0.032 | Tree loss: 2.868 | Accuracy: 0.154839 | 4.837 sec/iter
Average sp

Epoch: 93 | Batch: 000 / 026 | Total loss: 3.043 | Reg loss: 0.031 | Tree loss: 3.043 | Accuracy: 0.115234 | 4.816 sec/iter
Epoch: 93 | Batch: 001 / 026 | Total loss: 3.011 | Reg loss: 0.031 | Tree loss: 3.011 | Accuracy: 0.101562 | 4.816 sec/iter
Epoch: 93 | Batch: 002 / 026 | Total loss: 2.975 | Reg loss: 0.031 | Tree loss: 2.975 | Accuracy: 0.111328 | 4.816 sec/iter
Epoch: 93 | Batch: 003 / 026 | Total loss: 3.018 | Reg loss: 0.031 | Tree loss: 3.018 | Accuracy: 0.123047 | 4.815 sec/iter
Epoch: 93 | Batch: 004 / 026 | Total loss: 2.980 | Reg loss: 0.031 | Tree loss: 2.980 | Accuracy: 0.140625 | 4.815 sec/iter
Epoch: 93 | Batch: 005 / 026 | Total loss: 2.980 | Reg loss: 0.031 | Tree loss: 2.980 | Accuracy: 0.117188 | 4.814 sec/iter
Epoch: 93 | Batch: 006 / 026 | Total loss: 2.947 | Reg loss: 0.031 | Tree loss: 2.947 | Accuracy: 0.126953 | 4.814 sec/iter
Epoch: 93 | Batch: 007 / 026 | Total loss: 2.935 | Reg loss: 0.031 | Tree loss: 2.935 | Accuracy: 0.128906 | 4.814 sec/iter
Epoch: 9

Epoch: 95 | Batch: 009 / 026 | Total loss: 2.974 | Reg loss: 0.031 | Tree loss: 2.974 | Accuracy: 0.119141 | 4.794 sec/iter
Epoch: 95 | Batch: 010 / 026 | Total loss: 2.923 | Reg loss: 0.031 | Tree loss: 2.923 | Accuracy: 0.128906 | 4.793 sec/iter
Epoch: 95 | Batch: 011 / 026 | Total loss: 2.936 | Reg loss: 0.031 | Tree loss: 2.936 | Accuracy: 0.107422 | 4.793 sec/iter
Epoch: 95 | Batch: 012 / 026 | Total loss: 2.918 | Reg loss: 0.031 | Tree loss: 2.918 | Accuracy: 0.107422 | 4.793 sec/iter
Epoch: 95 | Batch: 013 / 026 | Total loss: 2.911 | Reg loss: 0.031 | Tree loss: 2.911 | Accuracy: 0.101562 | 4.792 sec/iter
Epoch: 95 | Batch: 014 / 026 | Total loss: 2.921 | Reg loss: 0.031 | Tree loss: 2.921 | Accuracy: 0.109375 | 4.792 sec/iter
Epoch: 95 | Batch: 015 / 026 | Total loss: 2.879 | Reg loss: 0.032 | Tree loss: 2.879 | Accuracy: 0.132812 | 4.792 sec/iter
Epoch: 95 | Batch: 016 / 026 | Total loss: 2.891 | Reg loss: 0.032 | Tree loss: 2.891 | Accuracy: 0.119141 | 4.791 sec/iter
Epoch: 9

Epoch: 97 | Batch: 018 / 026 | Total loss: 2.894 | Reg loss: 0.032 | Tree loss: 2.894 | Accuracy: 0.111328 | 4.772 sec/iter
Epoch: 97 | Batch: 019 / 026 | Total loss: 2.879 | Reg loss: 0.032 | Tree loss: 2.879 | Accuracy: 0.144531 | 4.772 sec/iter
Epoch: 97 | Batch: 020 / 026 | Total loss: 2.880 | Reg loss: 0.032 | Tree loss: 2.880 | Accuracy: 0.119141 | 4.772 sec/iter
Epoch: 97 | Batch: 021 / 026 | Total loss: 2.892 | Reg loss: 0.032 | Tree loss: 2.892 | Accuracy: 0.115234 | 4.771 sec/iter
Epoch: 97 | Batch: 022 / 026 | Total loss: 2.853 | Reg loss: 0.032 | Tree loss: 2.853 | Accuracy: 0.128906 | 4.771 sec/iter
Epoch: 97 | Batch: 023 / 026 | Total loss: 2.922 | Reg loss: 0.032 | Tree loss: 2.922 | Accuracy: 0.101562 | 4.771 sec/iter
Epoch: 97 | Batch: 024 / 026 | Total loss: 2.864 | Reg loss: 0.032 | Tree loss: 2.864 | Accuracy: 0.117188 | 4.77 sec/iter
Epoch: 97 | Batch: 025 / 026 | Total loss: 2.883 | Reg loss: 0.032 | Tree loss: 2.883 | Accuracy: 0.096774 | 4.77 sec/iter
Average sp

In [31]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [33]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 11.991871921182266


# Extract Rules

# Accumulate samples in the leaves

In [34]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 4060


In [35]:
method = 'greedy'

In [36]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [37]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

============== Pattern 1 ==============
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
============== Pattern 25

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))



============== Pattern 623 ==============
============== Pattern 624 ==============
============== Pattern 625 ==============
============== Pattern 626 ==============
============== Pattern 627 ==============
============== Pattern 628 ==============
============== Pattern 629 ==============
============== Pattern 630 ==============
============== Pattern 631 ==============
============== Pattern 632 ==============
============== Pattern 633 ==============
============== Pattern 634 ==============
============== Pattern 635 ==============
============== Pattern 636 ==============
============== Pattern 637 ==============
============== Pattern 638 ==============
============== Pattern 639 ==============
============== Pattern 640 ==============
============== Pattern 641 ==============
============== Pattern 642 ==============
============== Pattern 643 ==============
============== Pattern 644 ==============
============== Pattern 645 ==============
============== Pattern 646 ======

============== Pattern 864 ==============
============== Pattern 865 ==============
============== Pattern 866 ==============
============== Pattern 867 ==============
============== Pattern 868 ==============
============== Pattern 869 ==============
============== Pattern 870 ==============
============== Pattern 871 ==============
============== Pattern 872 ==============
============== Pattern 873 ==============
============== Pattern 874 ==============
============== Pattern 875 ==============
============== Pattern 876 ==============
============== Pattern 877 ==============
============== Pattern 878 ==============
============== Pattern 879 ==============
============== Pattern 880 ==============
============== Pattern 881 ==============
============== Pattern 882 ==============
============== Pattern 883 ==============
============== Pattern 884 ==============
============== Pattern 885 ==============
============== Pattern 886 ==============
============== Pattern 887 =======

============== Pattern 1862 ==============
============== Pattern 1863 ==============
============== Pattern 1864 ==============
============== Pattern 1865 ==============
============== Pattern 1866 ==============
============== Pattern 1867 ==============
============== Pattern 1868 ==============
============== Pattern 1869 ==============
============== Pattern 1870 ==============
============== Pattern 1871 ==============
============== Pattern 1872 ==============
============== Pattern 1873 ==============
============== Pattern 1874 ==============
============== Pattern 1875 ==============
============== Pattern 1876 ==============
============== Pattern 1877 ==============
============== Pattern 1878 ==============
============== Pattern 1879 ==============
============== Pattern 1880 ==============
============== Pattern 1881 ==============
============== Pattern 1882 ==============
============== Pattern 1883 ==============
============== Pattern 1884 ==============
===========

============== Pattern 2862 ==============
============== Pattern 2863 ==============
============== Pattern 2864 ==============
============== Pattern 2865 ==============
============== Pattern 2866 ==============
============== Pattern 2867 ==============
============== Pattern 2868 ==============
============== Pattern 2869 ==============
============== Pattern 2870 ==============
============== Pattern 2871 ==============
============== Pattern 2872 ==============
============== Pattern 2873 ==============
============== Pattern 2874 ==============
============== Pattern 2875 ==============
============== Pattern 2876 ==============
============== Pattern 2877 ==============
============== Pattern 2878 ==============
============== Pattern 2879 ==============
============== Pattern 2880 ==============
============== Pattern 2881 ==============
============== Pattern 2882 ==============
============== Pattern 2883 ==============
============== Pattern 2884 ==============
===========

============== Pattern 3861 ==============
============== Pattern 3862 ==============
============== Pattern 3863 ==============
============== Pattern 3864 ==============
============== Pattern 3865 ==============
============== Pattern 3866 ==============
============== Pattern 3867 ==============
============== Pattern 3868 ==============
============== Pattern 3869 ==============
============== Pattern 3870 ==============
============== Pattern 3871 ==============
============== Pattern 3872 ==============
============== Pattern 3873 ==============
============== Pattern 3874 ==============
============== Pattern 3875 ==============
============== Pattern 3876 ==============
============== Pattern 3877 ==============
============== Pattern 3878 ==============
============== Pattern 3879 ==============
============== Pattern 3880 ==============
============== Pattern 3881 ==============
============== Pattern 3882 ==============
============== Pattern 3883 ==============
===========

std comprehensibility: 4.078338748915884
var comprehensibility: 16.63284695090878
minimum comprehensibility: 42
maximum comprehensibility: 70
